# Advent of Code

## 2023-012-003
## 2023 003

https://adventofcode.com/2023/day/3

In [2]:
# Helper function to check if a position is valid in the grid
def is_valid_position(x, y, grid):
    return 0 <= x < len(grid) and 0 <= y < len(grid[0])

# Function to calculate the sum of part numbers based on their bounding box containing a symbol
def calculate_sum_with_bounding_box(grid):
    total_sum = 0

    for i in range(len(grid)):
        j = 0
        while j < len(grid[0]):
            # Check if the current cell starts a multi-digit number
            if grid[i][j].isdigit():
                number = ""
                start_j = j
                # Collect the entire number by traversing horizontally
                while j < len(grid[0]) and grid[i][j].isdigit():
                    number += grid[i][j]
                    j += 1

                # Determine the bounding box
                bounding_box_has_symbol = False
                for dx in range(-1, 2):  # Rows in the bounding box
                    for dy in range(-1, len(number) + 1):  # Columns in the bounding box
                        nx, ny = i + dx, start_j + dy
                        if is_valid_position(nx, ny, grid) and grid[nx][ny] not in "0123456789.":
                            bounding_box_has_symbol = True
                            break
                    if bounding_box_has_symbol:
                        break

                # If the bounding box contains a symbol, add the number to the total sum
                if bounding_box_has_symbol:
                    total_sum += int(number)
            else:
                j += 1  # Move to the next cell if not a digit
    return total_sum

# Function to parse input from a file
def parse_input(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file.readlines()]

# File paths
sample_file_path = 'sample-input.txt'
input_file_path = 'input.txt'

# Read inputs
sample_input = parse_input(sample_file_path)
actual_input = parse_input(input_file_path)

# Calculate sums
sample_result = calculate_sum_with_bounding_box(sample_input)
actual_result = calculate_sum_with_bounding_box(actual_input)

# Output results
print("Sample Input Sum:", sample_result)
print("Actual Input Sum:", actual_result)

Sample Input Sum: 4361
Actual Input Sum: 550934


In [9]:
# Function to find positions of stars
def find_star_positions(grid):
    return [(i, j) for i in range(len(grid)) for j in range(len(grid[0])) if grid[i][j] == '*']
# Function to find bounding boxes of stars
def find_bounding_boxes(star_positions):
    bounding_boxes = {}
    for star in star_positions:
        i, j = star
        bounding_boxes[star] = [(i + dx, j + dy) for dx in range(-1, 2) for dy in range(-1, 2)]
    return bounding_boxes
# Function to combine bounding boxes into a single bounding box
def combine_bounding_boxes(*boxes):
    min_row = min(box[0] for box in boxes)
    max_row = max(box[1] for box in boxes)
    min_col = min(box[2] for box in boxes)
    max_col = max(box[3] for box in boxes)
    return min_row, max_row, min_col, max_col
# Updated function to compute a bounding box for just the number positions
def compute_number_bounding_box(positions, grid):
    # Compute the smallest rectangle that includes all positions
    min_row = max(min(pos[0] for pos in positions) - 1, 0)  # Extend by 1 row above
    max_row = min(max(pos[0] for pos in positions) + 1, len(grid) - 1)  # Extend by 1 row below
    min_col = max(min(pos[1] for pos in positions) - 1, 0)  # Extend by 1 column to the left
    max_col = min(max(pos[1] for pos in positions) + 1, len(grid[0]) - 1)  # Extend by 1 column to the right
    return min_row, max_row, min_col, max_col
# Function to map numbers to their positions
def map_numbers_to_positions(grid):
    number_to_positions = {}
    for i in range(len(grid)):
        j = 0
        while j < len(grid[0]):
            if grid[i][j].isdigit():
                number = ""
                start_j = j
                positions = []
                while j < len(grid[0]) and grid[i][j].isdigit():
                    number += grid[i][j]
                    positions.append((i, j))
                    j += 1
                number_to_positions[number] = positions
            else:
                j += 1
    return number_to_positions
# Function to compute the bounding box for a set of positions
def compute_bounding_box(positions, grid):
    min_row = max(min(pos[0] for pos in positions), 0)  # Ensure it doesn't go outside the grid
    max_row = min(max(pos[0] for pos in positions), len(grid) - 1)
    min_col = max(min(pos[1] for pos in positions), 0)  # Ensure it doesn't go outside the grid
    max_col = min(max(pos[1] for pos in positions), len(grid[0]) - 1)
    return min_row, max_row, min_col, max_col
# Function to extract and print the area defined by a bounding box
def print_bounding_box(grid, bounding_box):
    min_row, max_row, min_col, max_col = bounding_box
    area = [grid[row][min_col:max_col + 1] for row in range(min_row, max_row + 1)]
    for line in area:
        print(line)
# Updated function to find stars with two adjacent numbers and combine bounding boxes
def find_and_combine_bounding_boxes(grid):
    star_positions = find_star_positions(grid)
    bounding_boxes = find_bounding_boxes(star_positions)
    number_to_positions = map_numbers_to_positions(grid)

    results = []

    for star, box in bounding_boxes.items():
        overlapping_numbers = []
        for number, positions in number_to_positions.items():
            if any(pos in box for pos in positions):
                overlapping_numbers.append((number, positions))

        # Check if there are exactly two adjacent numbers
        if len(overlapping_numbers) == 2:
            num1, positions1 = overlapping_numbers[0]
            num2, positions2 = overlapping_numbers[1]
            num1, num2 = int(num1), int(num2)
            gear_ratio = num1 * num2

            # Compute the bounding boxes for the numbers and star
            bbox_star = compute_bounding_box(box, grid)
            bbox1 = compute_number_bounding_box(positions1, grid)
            bbox2 = compute_number_bounding_box(positions2, grid)

            # Combine the bounding boxes
            combined_bbox = combine_bounding_boxes(bbox_star, bbox1, bbox2)

            results.append({
                "num1": num1,
                "num2": num2,
                "gear_ratio": gear_ratio,
                "combined_bbox": combined_bbox
            })

    # Print the results
    for result in results:
        print(f"Numbers: {result['num1']} and {result['num2']}")
        print(f"Gear Ratio: {result['gear_ratio']}")
        print("Combined Bounding Box:")
        print_bounding_box(grid, result['combined_bbox'])
        print("\n")

# Apply the function to the sample input
print("Sample Input Gear Details with Combined Bounding Boxes:")
find_and_combine_bounding_boxes(sample_input)

Sample Input Gear Details with Combined Bounding Boxes:
Numbers: 467 and 35
Gear Ratio: 16345
Combined Bounding Box:
467..
...*.
..35.
.....


Numbers: 755 and 598
Gear Ratio: 451490
Combined Bounding Box:
2.....
..755.
.*....
.598..




In [10]:
# Function to print the combined bounding boxes for the actual input
def print_combined_bounding_boxes(grid):
    star_positions = find_star_positions(grid)
    bounding_boxes = find_bounding_boxes(star_positions)
    number_to_positions = map_numbers_to_positions(grid)

    results = []

    for star, box in bounding_boxes.items():
        overlapping_numbers = []
        for number, positions in number_to_positions.items():
            if any(pos in box for pos in positions):
                overlapping_numbers.append((number, positions))

        # Check if there are exactly two adjacent numbers
        if len(overlapping_numbers) == 2:
            num1, positions1 = overlapping_numbers[0]
            num2, positions2 = overlapping_numbers[1]
            num1, num2 = int(num1), int(num2)
            gear_ratio = num1 * num2

            # Compute the bounding boxes for the numbers and star
            bbox_star = compute_bounding_box(box, grid)
            bbox1 = compute_number_bounding_box(positions1, grid)
            bbox2 = compute_number_bounding_box(positions2, grid)

            # Combine the bounding boxes
            combined_bbox = combine_bounding_boxes(bbox_star, bbox1, bbox2)

            results.append({
                "num1": num1,
                "num2": num2,
                "gear_ratio": gear_ratio,
                "combined_bbox": combined_bbox
            })

    # Print the results
    for result in results:
        print(f"Numbers: {result['num1']} and {result['num2']}")
        print(f"Gear Ratio: {result['gear_ratio']}")
        print("Combined Bounding Box:")
        print_bounding_box(grid, result['combined_bbox'])
        print("\n")

# Example code for actual input (commented to prevent long output when executed)

print("Actual Input Gear Details with Combined Bounding Boxes:")
print_combined_bounding_boxes(actual_input)



Actual Input Gear Details with Combined Bounding Boxes:
Numbers: 817 and 427
Gear Ratio: 348859
Combined Bounding Box:
.........
.817..336
....*....
.....427.
776......


Numbers: 441 and 760
Gear Ratio: 335160
Combined Bounding Box:
.........
.441*760.
.........


Numbers: 238 and 573
Gear Ratio: 136374
Combined Bounding Box:
......
.238..
.*....
..573.
......


Numbers: 324 and 807
Gear Ratio: 261468
Combined Bounding Box:
.......
.324..*
....*..
...807.
.......


Numbers: 482 and 275
Gear Ratio: 132550
Combined Bounding Box:
......
.482.9
.*....
..275.
......


Numbers: 917 and 409
Gear Ratio: 375053
Combined Bounding Box:
........
....917.
....*...
.409....
........


Numbers: 358 and 730
Gear Ratio: 261340
Combined Bounding Box:
.....$9
.358...
....*..
...730.
.......


Numbers: 86 and 340
Gear Ratio: 29240
Combined Bounding Box:
.....85
.86....
...*...
...340.
.......


Numbers: 957 and 484
Gear Ratio: 463188
Combined Bounding Box:
........
....957.
....*...
.484...@
.......3


N

In [15]:
# Function to identify part numbers based on adjacency to symbols
def identify_part_numbers(grid):
    part_number_positions = set()
    directions = [(-1, -1), (-1, 0), (-1, 1),  # Top-left, top, top-right
                  (0, -1),          (0, 1),    # Left, right
                  (1, -1), (1, 0), (1, 1)]    # Bottom-left, bottom, bottom-right

    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j].isdigit():
                is_part_number = False
                for dx, dy in directions:
                    nx, ny = i + dx, j + dy
                    if is_valid_position(nx, ny, grid) and grid[nx][ny] not in "0123456789.":
                        is_part_number = True
                        break
                if is_part_number:
                    part_number_positions.add((i, j))
    return part_number_positions

# Update the function to restrict adjacent numbers to part numbers
def find_and_combine_bounding_boxes_restricted(grid):
    my_sum=0
    star_positions = find_star_positions(grid)
    bounding_boxes = find_bounding_boxes(star_positions)
    number_to_positions = map_numbers_to_positions(grid)
    part_number_positions = identify_part_numbers(grid)

    results = []

    for star, box in bounding_boxes.items():
        overlapping_numbers = []
        for number, positions in number_to_positions.items():
            # Check if the number positions overlap with the star's bounding box
            # and are part numbers
            if any(pos in box and pos in part_number_positions for pos in positions):
                overlapping_numbers.append((number, positions))

        # Check if there are exactly two adjacent part numbers
        if len(overlapping_numbers) == 2:
            num1, positions1 = overlapping_numbers[0]
            num2, positions2 = overlapping_numbers[1]
            num1, num2 = int(num1), int(num2)
            gear_ratio = num1 * num2
            my_sum+=gear_ratio
            # Compute the bounding boxes for the numbers and star
            bbox_star = compute_bounding_box(box, grid)
            bbox1 = compute_number_bounding_box(positions1, grid)
            bbox2 = compute_number_bounding_box(positions2, grid)

            # Combine the bounding boxes
            combined_bbox = combine_bounding_boxes(bbox_star, bbox1, bbox2)

            results.append({
                "num1": num1,
                "num2": num2,
                "gear_ratio": gear_ratio,
                "combined_bbox": combined_bbox
            })
    print(my_sum)
    # Print the results
    for result in results:
        
        print(f"Numbers: {result['num1']} and {result['num2']}")
        print(f"Gear Ratio: {result['gear_ratio']}")
        print("Combined Bounding Box:")
        print_bounding_box(grid, result['combined_bbox'])
        print("\n")

# Example code for actual input (commented to prevent long output when executed)

print("Actual Input Gear Details with Restricted Part Numbers:")
find_and_combine_bounding_boxes_restricted(actual_input)


Actual Input Gear Details with Restricted Part Numbers:
32337774
Numbers: 817 and 427
Gear Ratio: 348859
Combined Bounding Box:
.........
.817..336
....*....
.....427.
776......


Numbers: 441 and 760
Gear Ratio: 335160
Combined Bounding Box:
.........
.441*760.
.........


Numbers: 238 and 573
Gear Ratio: 136374
Combined Bounding Box:
......
.238..
.*....
..573.
......


Numbers: 324 and 807
Gear Ratio: 261468
Combined Bounding Box:
.......
.324..*
....*..
...807.
.......


Numbers: 482 and 275
Gear Ratio: 132550
Combined Bounding Box:
......
.482.9
.*....
..275.
......


Numbers: 917 and 409
Gear Ratio: 375053
Combined Bounding Box:
........
....917.
....*...
.409....
........


Numbers: 358 and 730
Gear Ratio: 261340
Combined Bounding Box:
.....$9
.358...
....*..
...730.
.......


Numbers: 86 and 340
Gear Ratio: 29240
Combined Bounding Box:
.....85
.86....
...*...
...340.
.......


Numbers: 957 and 484
Gear Ratio: 463188
Combined Bounding Box:
........
....957.
....*...
.484...@
...

In [16]:
# Function to find stars with an incorrect number of adjacent numbers and combine bounding boxes
def find_stars_with_incorrect_adjacent_numbers(grid):
    star_positions = find_star_positions(grid)
    bounding_boxes = find_bounding_boxes(star_positions)
    number_to_positions = map_numbers_to_positions(grid)
    part_number_positions = identify_part_numbers(grid)

    results = []

    for star, box in bounding_boxes.items():
        overlapping_numbers = []
        for number, positions in number_to_positions.items():
            # Check if the number positions overlap with the star's bounding box
            # and are part numbers
            if any(pos in box and pos in part_number_positions for pos in positions):
                overlapping_numbers.append((number, positions))

        # Check if the number of adjacent part numbers is not 2
        if len(overlapping_numbers) != 2 and len(overlapping_numbers) > 0:
            # Collect details for further analysis
            details = {
                "star_position": star,
                "adjacent_numbers": overlapping_numbers,
                "count_adjacent": len(overlapping_numbers),
                "bounding_box": compute_bounding_box(box, grid)
            }
            results.append(details)

    # Print the results
    for result in results:
        print(f"Star Position: {result['star_position']}")
        print(f"Count of Adjacent Numbers: {result['count_adjacent']}")
        print("Adjacent Numbers:")
        for num, positions in result['adjacent_numbers']:
            print(f"  Number: {num}, Positions: {positions}")
        print("Bounding Box Around Star:")
        print_bounding_box(grid, result['bounding_box'])
        print("\n")

# Example code for actual input (commented to prevent long output when executed)

print("Stars with Incorrect Number of Adjacent Numbers:")
find_stars_with_incorrect_adjacent_numbers(actual_input)



Stars with Incorrect Number of Adjacent Numbers:
Star Position: (2, 63)
Count of Adjacent Numbers: 1
Adjacent Numbers:
  Number: 548, Positions: [(3, 64), (3, 65), (3, 66)]
Bounding Box Around Star:
...
5*.
..5


Star Position: (4, 82)
Count of Adjacent Numbers: 1
Adjacent Numbers:
  Number: 748, Positions: [(3, 83), (3, 84), (3, 85)]
Bounding Box Around Star:
..7
.*.
..4


Star Position: (5, 12)
Count of Adjacent Numbers: 1
Adjacent Numbers:
  Number: 368, Positions: [(4, 13), (4, 14), (4, 15)]
Bounding Box Around Star:
..3
.*.
882


Star Position: (5, 51)
Count of Adjacent Numbers: 1
Adjacent Numbers:
  Number: 555, Positions: [(6, 51), (6, 52), (6, 53)]
Bounding Box Around Star:
.88
.*.
.55


Star Position: (5, 71)
Count of Adjacent Numbers: 1
Adjacent Numbers:
  Number: 135, Positions: [(4, 70), (4, 71), (4, 72)]
Bounding Box Around Star:
135
.*.
..9


Star Position: (7, 28)
Count of Adjacent Numbers: 1
Adjacent Numbers:
  Number: 776, Positions: [(6, 29), (6, 30), (6, 31)]
Boundin

In [25]:
# Updated function to map positions to their respective numbers
def create_position_to_number_map(number_to_positions):
    position_to_number = {}
    for number, positions in number_to_positions.items():
        for pos in positions:
            position_to_number[pos] = int(number)  # Map the position to the number
    return position_to_number

# Function to find stars and associate them with numbers in adjacent positions
def find_stars_with_two_numbers(grid):
    sum=0
    star_positions = find_star_positions(grid)
    number_to_positions = map_numbers_to_positions(grid)
    part_number_positions = identify_part_numbers(grid)
    position_to_number = create_position_to_number_map(number_to_positions)

    results = []

    for star in star_positions:
        i, j = star
        # Get the 8 surrounding positions
        surrounding_positions = [
            (i + dx, j + dy)
            for dx, dy in [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
            if is_valid_position(i + dx, j + dy, grid)
        ]

        # Find unique numbers corresponding to those positions
        adjacent_numbers = {
            position_to_number[pos]
            for pos in surrounding_positions
            if pos in position_to_number and pos in part_number_positions
        }

        # Check if exactly two unique numbers are adjacent
        if len(adjacent_numbers) == 2:
            num1, num2 = sorted(adjacent_numbers)
            gear_ratio = num1 * num2
            sum+=gear_ratio
            results.append({
                "star_position": star,
                "num1": num1,
                "num2": num2,
                "gear_ratio": gear_ratio
            })
    print(sum)
    print(len(results))
    # Print the results
    # for result in results:
    #     print(f"Star Position: {result['star_position']}")
    #     print(f"Numbers: {result['num1']} and {result['num2']}")
    #     print(f"Gear Ratio: {result['gear_ratio']}\n")
    
# Example code for actual input (commented to prevent long output when executed)

print("Stars with Exactly Two Adjacent Numbers:")
find_stars_with_two_numbers(actual_input)


Stars with Exactly Two Adjacent Numbers:
32337774
129


In [24]:
# Refined function to find stars and associate them with numbers correctly
def find_stars_with_two_numbers_refined(grid):
    star_positions = find_star_positions(grid)
    number_to_positions = map_numbers_to_positions(grid)
    part_number_positions = identify_part_numbers(grid)
    position_to_number = create_position_to_number_map(number_to_positions)

    results = []

    for star in star_positions:
        i, j = star
        # Get the 8 surrounding positions
        surrounding_positions = [
            (i + dx, j + dy)
            for dx, dy in [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
            if is_valid_position(i + dx, j + dy, grid)
        ]

        # Find unique numbers corresponding to those positions
        adjacent_numbers = {}
        for pos in surrounding_positions:
            if pos in position_to_number and pos in part_number_positions:
                num = position_to_number[pos]
                if num not in adjacent_numbers:
                    adjacent_numbers[num] = []
                adjacent_numbers[num].append(pos)

        # Check if exactly two unique numbers are adjacent
        if len(adjacent_numbers) == 2:
            num1, num2 = sorted(adjacent_numbers.keys())
            gear_ratio = num1 * num2

            results.append({
                "star_position": star,
                "num1": num1,
                "num2": num2,
                "gear_ratio": gear_ratio,
                "positions_num1": adjacent_numbers[num1],
                "positions_num2": adjacent_numbers[num2],
            })
    print(len(results))
    # Print the results
    # for result in results:
    #     print(f"Star Position: {result['star_position']}")
    #     print(f"Numbers: {result['num1']} and {result['num2']}")
    #     print(f"Gear Ratio: {result['gear_ratio']}")
    #     print(f"Positions for {result['num1']}: {result['positions_num1']}")
    #     print(f"Positions for {result['num2']}: {result['positions_num2']}\n")

# Example code for actual input (commented to prevent long output when executed)

print("Stars with Exactly Two Adjacent Numbers (Refined):")
find_stars_with_two_numbers_refined(actual_input)


Stars with Exactly Two Adjacent Numbers (Refined):
129
